# ACLED API User Guide:
https://acleddata.com/acleddatanew/wp-content/uploads/2020/10/ACLED_API-User-Guide_2020.pdf

In [43]:
import pandas as pd

import csv
import requests
import json


def get_acled_dataframe(url):

    with requests.Session() as s:
        download = s.get(url)

    decoded_content = download.content.decode('utf-8')

    res = json.loads(decoded_content)
    data = res['data']

    return pd.DataFrame(data)

In [58]:
url = 'https://api.acleddata.com/acled/read?key=fTIpbk9YOV5PYNcjf3v3&email=bwilki13@gmu.edu&limit=100&event_date=2022-07-04'
df = get_acled_dataframe(url)

99

In [57]:

# date (str): YYYY-MM-DD
# date_range (str): YYYY-MM-DD:AND:YYYY-MM-DD

def get_url(limit=100, date=None, date_range=None, location=None):
    
    email = 'bwilki13@gmu.edu'
    acled_key = 'fTIpbk9YOV5PYNcjf3v3'
    
    url = f'https://api.acleddata.com/acled/read?key={acled_key}&email={email}'
    
    filters = f'&limit={limit}'
    if date:
        filters += f'&event_date={date}'
    if date_range:
        filters += f'&event_date:BETWEEN:{date_range}'
    
    return url + filters

get_url(1000)

'https://api.acleddata.com/acled/read?key=fTIpbk9YOV5PYNcjf3v3&email=bwilki13@gmu.edu&limit=1000&event_date=2022-07-04'

<code>

# key = {api_key}
# email = {email address associated with key}
# data_id = ?data_id={number}
# iso = ?iso={number}
# event_id_cnty LIKE ?event_id_cnty={text}
# event_id_no_cnty LIKE ?event_id_no_cnty={text}
# event_date = ?event_date={yyyy-mm-dd}
# year = ?year={yyyy}
# time_precision = ?time_precision={number}
# event_type LIKE ?event_type={text}
# sub_event_type LIKE ?sub_event_type={text}
# actor1 LIKE ?actor1={text}
# assoc_actor_1 LIKE ?assoc_actor_1={text}
# inter1 = ?inter1={number}
# actor2 LIKE ?actor2={text}
# assoc_actor_2 LIKE ?assoc_actor_2={text}
# inter2 = ?inter2={number}
# interaction = ?interaction={number}
# region = ?region={number}
# country LIKE ?country={text}
# admin1 LIKE ?admin1={text}
# admin2 LIKE ?admin2={text}
    
<code>